#Импорт Библиотек и настройка профиля kaggle

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import heapq
import math
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, TargetEncoder, OrdinalEncoder
from sklearn.metrics import classification_report, make_scorer, f1_score, mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import GridSearchCV


In [2]:
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


#Классификация

##Подготовка данных



###Скачивание датасета

In [3]:
!kaggle datasets download -d shrutimechlearn/churn-modelling
!unzip churn-modelling

Dataset URL: https://www.kaggle.com/datasets/shrutimechlearn/churn-modelling
License(s): unknown
churn-modelling.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  churn-modelling.zip
replace Churn_Modelling.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Churn_Modelling.csv     


###Исходные данные

In [4]:
df = pd.read_csv("Churn_Modelling.csv")
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Удалим идентификаторы не несущие полезной информации

In [6]:
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1,inplace=True)

###Проверка наличия нулевых значений

In [7]:
df.isnull().sum()

,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0
HasCrCard,0
IsActiveMember,0
EstimatedSalary,0


###Обработка категориальных признаков

Проверим количество уникальных значений категориальных признаков для выбора подходящего метода кодировки

In [8]:
df.select_dtypes(include=["object"]).nunique()

,0
Geography,3
Gender,2


Так как категориальные признаки имеют маленькое количество уникальных значений применим кодировку One-Hot

In [9]:
encoder = OneHotEncoder(sparse_output=False)
categorical_columns = df.select_dtypes(include=['object']).columns.to_list()
encoded = pd.DataFrame(encoder.fit_transform(df[categorical_columns]), columns=encoder.get_feature_names_out(categorical_columns))
df = pd.concat([df, encoded], axis=1)
df.drop(categorical_columns, axis=1, inplace=True)

###Разделим датасет на train test и validation

In [10]:
X_train_val, X_test, Y_train_val, Y_test = train_test_split(df.drop(columns="Exited"), df["Exited"], test_size = 0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size = 0.2)

Так как Датасет имеет очень большой размер и моя  простая модель крайне нееффективна по времени, обучение будеn происходить на меньшей выборке из датасета

In [ ]:
idx = np.random.choice(X_train.shape[0], 1000, replace = False)
Y_train = np.asarray(Y_train).flatten()
X_train, Y_train = X_train[idx,:], Y_train[idx]

Сохраним данные обучения бейзлайна

In [11]:
X_train_baseline, X_test_baseline, Y_train_baseline, Y_test_baseline = X_train, X_test, Y_train, Y_test

##Бейзлайн


In [12]:
classifier = GradientBoostingClassifier()
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1615
           1       0.78      0.48      0.60       385

    accuracy                           0.87      2000
   macro avg       0.83      0.72      0.76      2000
weighted avg       0.87      0.87      0.86      2000



##Улучшение бейзлайна
Подбор гиперпараметров на полном датасете нецелесообраззен так как может занимать очень долгое время, а подбор гиперпараметров на неполном датасете не повышают качество предсказаний, поэтому сравнение будет проводиться между байзланом и моеймоделью в условиях байзлайна.\

##Собственная реализация

In [13]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class MyDecisionTreeRegressor:
    def __init__(self, max_depth=5, min_samples_split=2, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
        self.max_features = max_features

    def mse(self, y):
        if len(y) == 0:
            return 0.0
        return np.mean((y - y.mean()) ** 2)

    def best_split(self, X, y):
        n_samples, n_features = X.shape
        best_feature = None
        best_threshold = None
        best_gain = 0
        parent_mse = self.mse(y)

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for t in thresholds:
                left = X[:, feature] <= t
                right = ~left

                if np.sum(left) == 0 or np.sum(right) == 0:
                    continue

                left_y = y[left]
                right_y = y[right]

                mse_left = self.mse(left_y)
                mse_right = self.mse(right_y)

                w_left = len(left_y) / n_samples
                w_right = len(right_y) / n_samples

                split_mse = w_left * mse_left + w_right * mse_right
                gain = parent_mse - split_mse

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = t

        return best_feature, best_threshold, best_gain

    def build(self, X, y, depth):
        if (depth >= self.max_depth or len(y) < self.min_samples_split or len(np.unique(y)) == 1):
            leaf_value = np.mean(y)
            return Node(value=leaf_value)

        feature, threshold, gain = self.best_split(X, y)

        if gain == 0:
            leaf_value = np.mean(y)
            return Node(value=leaf_value)

        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        left_child = self.build(X[left_mask], y[left_mask], depth + 1)
        right_child = self.build(X[right_mask], y[right_mask], depth + 1)

        return Node(feature, threshold, left_child, right_child)

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.root = self.build(np.array(X), np.array(y), depth=0)

    def predict_one(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self.predict_one(x, node.left)
        else:
            return self.predict_one(x, node.right)


    def predict(self, X):
        X = np.asarray(X)
        return np.array([self.predict_one(x, self.root) for x in X])



class MyGradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=10, min_samples_split=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []
        self.init_log_odds = None

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        p = np.clip(y.mean(), 1e-6, 1 - 1e-6)
        self.init_log_odds = np.log(p / (1 - p))

        current_pred = np.full(len(y), self.init_log_odds)

        for _ in range(self.n_estimators):
            residuals = y - self.sigmoid(current_pred)

            tree = MyDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            tree.fit(X, residuals)
            self.trees.append(tree)

            update = tree.predict(X)
            current_pred += self.learning_rate * update

    def predict_proba(self, X):
        X = np.asarray(X)
        pred = np.full(len(X), self.init_log_odds)

        for tree in self.trees:
            pred += self.learning_rate * tree.predict(X)

        prob = self.sigmoid(pred)
        return np.vstack([1 - prob, prob]).T

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)

In [14]:
classifier = MyGradientBoostingClassifier()
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.88      0.95      0.92      1615
           1       0.70      0.47      0.56       385

    accuracy                           0.86      2000
   macro avg       0.79      0.71      0.74      2000
weighted avg       0.85      0.86      0.85      2000



- Моя модель f1[1] = 0.56  
- sklearn f1[1] = 0.60


#Регрессия

##Подготовка данных

###Скачивание датасета


In [15]:
!kaggle datasets download -d jakewright/house-price-data
!unzip house-price-data.zip

Dataset URL: https://www.kaggle.com/datasets/jakewright/house-price-data
License(s): other
  0% 0.00/48.5M [00:00<?, ?B/s]
100% 48.5M/48.5M [00:00<00:00, 1.56GB/s]
Archive:  house-price-data.zip
  inflating: kaggle_london_house_price_data.csv  
  inflating: kaggle_london_house_price_data.parquet  


###Исходные данные

In [16]:
reg_df = pd.read_csv("kaggle_london_house_price_data.csv")
reg_df

,fullAddress,postcode,country,outcode,latitude,longitude,bathrooms,bedrooms,floorAreaSqM,livingRooms,...,saleEstimate_upperPrice,saleEstimate_confidenceLevel,saleEstimate_ingestedAt,saleEstimate_valueChange.numericChange,saleEstimate_valueChange.percentageChange,saleEstimate_valueChange.saleDate,history_date,history_price,history_percentageChange,history_numericChange
0,"Flat 9, 35 Furnival Street, London, EC4A 1JQ",EC4A 1JQ,England,EC4A,51.517282,-0.110314,1.0,1.0,45.0,1.0,...,630000.0,HIGH,2024-10-07T13:26:59.894Z,244000.0,68.539326,2010-03-30,1995-01-02,830000,NaN,NaN
1,"Flat 6, 35 Furnival Street, London, EC4A 1JQ",EC4A 1JQ,England,EC4A,51.517282,-0.110314,NaN,NaN,NaN,NaN,...,660000.0,MEDIUM,2024-10-07T13:26:59.894Z,425000.0,242.857143,2000-05-26,1995-01-02,830000,NaN,NaN
2,"Flat 35, Octavia House, Medway Street, London,...",SW1P 2TA,England,SW1P,51.495505,-0.132379,2.0,2.0,71.0,1.0,...,834000.0,MEDIUM,2025-01-10T11:04:57.114Z,49000.0,6.901408,2019-09-04,1995-01-03,249950,NaN,NaN
3,"24 Chester Court, Lomond Grove, London, SE5 7HS",SE5 7HS,England,SE5,51.478185,-0.092201,1.0,1.0,64.0,1.0,...,407000.0,HIGH,2024-10-07T13:26:59.894Z,28000.0,7.777778,2024-01-25,1995-01-03,32000,NaN,NaN
4,"18 Alexandra Gardens, London, N10 3RL",N10 3RL,England,N10,51.588774,-0.139599,1.0,4.0,137.0,2.0,...,1324000.0,HIGH,2024-10-07T13:26:59.894Z,81000.0,6.864407,2022-12-14,1995-01-03,133000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418196,"9 Harvard Road, London, SE13 6SE",SE13 6SE,England,SE13,51.452919,-0.010221,2.0,3.0,123.0,2.0,...,711000.0,HIGH,2025-01-10T11:04:57.114Z,27000.0,4.153846,2024-09-27,2024-09-27,650000,17.117117,95000.0
418197,"Lower Ground Floor Flat 5, Northwood Hall, Hor...",N6 5PE,England,N6,51.571692,-0.137007,1.0,2.0,60.0,1.0,...,473000.0,HIGH,2025-01-10T11:04:57.114Z,10000.0,2.272727,2024-09-27,2024-09-27,440000,2.325581,10000.0
418198,"17 Merritt Road, London, SE4 1DU",SE4 1DU,England,SE4,51.456054,-0.035073,2.0,3.0,100.0,1.0,...,853000.0,HIGH,2025-01-10T11:04:57.114Z,38000.0,4.909561,2024-09-27,2024-09-27,774000,128.655835,435500.0
418199,"15 Chester Row, London, SW1W 9JF",SW1W 9JF,England,SW1W,51.493587,-0.152122,3.0,4.0,218.0,2.0,...,5975000.0,MEDIUM,2025-01-10T11:04:57.114Z,-818000.0,-13.088000,2024-09-27,2024-09-27,6250000,73.611111,2650000.0


Удалим нижние и верхние оценки цены и оценки аренды для избежания утечек данных. Также удалим столбцы указывающие на адрес, так как в датасете присктствуют стодбцы "lattitude"  и "longitude" более точно указывающие на местоположение дома.

In [17]:
reg_df = reg_df.drop(["rentEstimate_lowerPrice", "rentEstimate_currentPrice", "rentEstimate_upperPrice", "saleEstimate_lowerPrice", "saleEstimate_upperPrice", "country", "fullAddress", "postcode", "outcode"], axis = 1)

###Проверим наличие нулевых значений и удалим содержащие и стоки
Набор данных очень большой, поэтому мы можем позволить себе удалить любые строки с нулевыми значениями без значительной потери данных.

In [18]:
reg_df.isnull().sum()

,0
latitude,0
longitude,0
bathrooms,77931
bedrooms,40536
floorAreaSqM,25210
livingRooms,60500
tenure,11512
propertyType,1136
currentEnergyRating,84526
saleEstimate_currentPrice,640


In [19]:
reg_df.dropna(how='any', axis=0, inplace=True)
reg_df.reset_index(drop=True, inplace=True)

###Кодирование категориальных признаков

Посмотрим количество уникальных значений для определения подходящего метода

In [20]:
reg_df.select_dtypes(include=["object"]).nunique()

,0
tenure,4
propertyType,19
currentEnergyRating,7
saleEstimate_confidenceLevel,3
saleEstimate_ingestedAt,4
saleEstimate_valueChange.saleDate,3497
history_date,7394


Для признаков с маленьким количеством уникалтных значений применим кодировку One-Hot

In [21]:
encoder = OneHotEncoder(sparse_output=False)
columns_to_encode = ["propertyType", "tenure"]
encoded = pd.DataFrame(encoder.fit_transform(reg_df.get(columns_to_encode)), columns=encoder.get_feature_names_out(columns_to_encode))
reg_df.drop(columns_to_encode, axis=1, inplace=True)
reg_df = pd.concat([reg_df, encoded], axis=1)

Для признаков, где порядок значений несет смысл применим порядковую кодировку.\
Посмотрим все возможные значения в столбцах и закодируем их по порядку

In [22]:
reg_df["currentEnergyRating"].value_counts()

,count
currentEnergyRating,
D,68747
C,62568
E,13928
B,11112
F,715
G,125
A,83


In [23]:
reg_df["saleEstimate_confidenceLevel"].value_counts()

,count
saleEstimate_confidenceLevel,
HIGH,132428
MEDIUM,20522
LOW,4328


In [24]:
encoder = OrdinalEncoder(categories=[['G', 'F', 'E', 'D', 'C', 'B', 'A'], ["LOW", "MEDIUM", "HIGH"]])
encoded = pd.DataFrame(encoder.fit_transform(reg_df.get(["currentEnergyRating","saleEstimate_confidenceLevel"])), columns=encoder.get_feature_names_out(["currentEnergyRating","saleEstimate_confidenceLevel"]))
reg_df["currentEnergyRating"] = encoded["currentEnergyRating"]
reg_df["saleEstimate_confidenceLevel"] = encoded["saleEstimate_confidenceLevel"]

Обработка даты и времени.\
Запмшем дату и время в целочисленном представлении

In [25]:
reg_df["history_date"] = pd.to_datetime(reg_df["history_date"]).astype("int64")
reg_df["saleEstimate_valueChange.saleDate"] = pd.to_datetime(reg_df["saleEstimate_valueChange.saleDate"]).astype("int64")
reg_df["saleEstimate_ingestedAt"] = pd.to_datetime(reg_df["saleEstimate_ingestedAt"]).astype("int64")

###Разделим данные на train test и validation

In [26]:
X_train_val, X_test, Y_train_val, Y_test = train_test_split(reg_df.drop("saleEstimate_currentPrice", axis=1), reg_df[["saleEstimate_currentPrice"]], test_size= 0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size= 0.2)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
X_val = np.asarray(X_val)
Y_train = np.asarray(Y_train).flatten()

##Бейзлайн

In [27]:
regressor = GradientBoostingRegressor()
regressor.fit(X_train, Y_train)
Y_pred = regressor.predict(X_test)

In [28]:
mae = mean_absolute_error(Y_pred, Y_test)
rmse = root_mean_squared_error(Y_pred, Y_test)
r2 = r2_score(Y_pred, Y_test)
print("MAE = ", mae)
print("MSE = ", rmse)
print("R2 = ", r2)

MAE =  87886.25414362825
MSE =  190241.70283835148
R2 =  0.933118762664302


##Улучшение бейзлайна
Подбор гиперпараметров на полном датасете нецелесообраззен так как может занимать очень долгое время, а подбор гиперпараметров на неполном датасете не повышают качество предсказаний, поэтому сравнение будет проводиться между байзланом и моеймоделью в условиях байзлайна.\

##Собственная реализация

In [29]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class MyDecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2, max_features=None):
        self.max_depth=max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features

    def _get_max_features(self, n_features):
        if self.max_features == None:
            return n_features
        if self.max_features == "sqrt":
            return max(1, int(np.sqrt(n_features)))
        if self.max_features == "log2":
            return max(1, int(np.log2(n_features)))
        return n_features

    def _find_best_feature(self, X, y):
        n_samples, n_features = X.shape


        features = np.random.choice(n_features, self._get_max_features(n_features), replace=False)

        parent_mse = mean_squared_error(y, np.full_like(y, y.mean()))
        best_threshold = None
        best_feature = None
        best_gain = 0
        for feature in features:
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] < threshold
                right_mask = X[:, feature] >= threshold

                if left_mask.sum() == 0 or right_mask.sum() == 0:
                    continue

                left = y[left_mask]
                right = y[right_mask]

                left_mean = np.mean(left)
                right_mean = np.mean(right)

                left_mse = mean_squared_error(left, np.full_like(left, left_mean))
                right_mse = mean_squared_error(right, np.full_like(right, right_mean))

                mse = left.size / n_samples * left_mse + right.size / n_samples * right_mse

                gain = parent_mse - mse

                if gain > best_gain:
                    best_gain = gain
                    best_threshold = threshold
                    best_feature = feature

        return best_feature, best_threshold, best_gain

    def _build(self, X, y, depth):
        n_samples, n_features = X.shape
        node = None

        if (self.max_depth is None or depth < self.max_depth) and n_samples >= self.min_samples_split:
            node = Node()
            feature, threshold, gain = self._find_best_feature(X, y)
            if feature is None or threshold is None or gain == 0:
                return Node(value=y.mean())

            right_mask = X[:, feature] >= threshold
            left_mask = X[:, feature] < threshold

            node.feature = feature
            node.threshold = threshold

            node.left  = self._build(X[left_mask], y[left_mask], depth+1)
            node.right = self._build(X[right_mask], y[right_mask], depth+1)
        else:
            node = Node(value=y.mean())

        return node


    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.root = self._build(X, y, depth=0)


    def _predict(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature] >= node.threshold:
            return self._predict(x, node.right)
        else:
            return self._predict(x, node.left)

    def predict(self, X):
        X = np.asarray(X)
        pred = np.array([self._predict(sample, self.root) for sample in X])
        return pred


class MyGradientBoostingRegressor:
    def __init__(self, n_trees=10, max_depth=None, min_samples_split=2, max_features="sqrt",learning_rate=0.1):
        self.n_trees=n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.learning_rate = learning_rate

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.base_pred = np.mean(y)
        self.trees = []

        pred = self.base_pred
        residuals = y - pred
        for i in range(self.n_trees):
            tree = MyDecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split, max_features=self.max_features)
            tree.fit(X, residuals)
            pred = tree.predict(X)
            self.trees.append(tree)
            residuals -= self.learning_rate * pred

    def predict(self, X):
        X = np.asarray(X)
        n_samples, _  = X.shape

        pred = np.full(n_samples, self.base_pred)

        for tree in self.trees:
            pred += self.learning_rate * tree.predict(X)

        return pred


In [32]:
regressor = MyGradientBoostingRegressor(n_trees=100)
regressor.fit(X_train, Y_train)

In [33]:
Y_pred = regressor.predict(X_test)
mae = mean_absolute_error(Y_pred, Y_test)
mse = mean_squared_error(Y_pred, Y_test)
r2 = r2_score(Y_pred, Y_test)
print("MAE = ", mae)
print("MSE = ", mse)
print("R2 = ", r2)

MAE =  149261.7515675776
MSE =  119096413172.9173
R2 =  0.6618218340023103


- Моя модель
    - MAE =  149261.7515675776
    - MSE =  119096413172.9173
    - R2 =  0.6618218340023103
- sklearn
    - MAE =  87886.25414362825
    - MSE =  190241.70283835148
    - R2 =  0.933118762664302